In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data = pd.read_csv('./test.csv', delimiter='|')
data.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,31747,Developer Security Operation (DevSecOps),Jakarta Pusat,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,Sarjana (S1),Penuh Waktu,"Komputer/Teknologi Informasi,IT-Admin Jaringan...","Tip;Asuransi kesehatan;Parkir;Waktu regular, S...",18 days,201 - 500 pekerja,Komputer/Teknik Informatika (Perangkat Lunak),1. Protect All Aspects of Your Source Cod...,7500000
1,31748,"Bancassurance Officer (Area: Bali, Sulawesi Ut...",Palu,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Penjualan / Pemasaran,Penjualan - Jasa Keuangan","Asuransi kesehatan;Waktu regular, Senin - Juma...",14 days,201 - 500 pekerja,Asuransi,Bancassurance Officer adalah Representative da...,4100000
2,31749,Marketing Staff,Jakarta Utara,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...",NaN,NaN,NaN,Manufaktur/Produksi,Bertanggung jawab atas penjualan sesuai dengan...,4875000
3,31750,Section Head Commercials,Jakarta Raya,IDR,Manajer/Asisten Manajer,3 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel",Uniform,NaN,501 - 1000 pekerja,Retail/Merchandise,Tanggung Jawab :Bertanggung jawab atas keselur...,6750000
4,31751,Social Media HEAD,Aceh,IDR,Supervisor/Koordinator,3 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Penjualan / Pemasaran,Digital Marketing",NaN,NaN,1- 50 pekerja,Makanan & Minuman/Katering/Restoran,# Memiliki pengalaman di atas# Harus memiliki ...,10680000


In [5]:
def preprocess_text(text):
    stop_words_english = set(stopwords.words('english'))
    stop_words_indonesian = set(stopwords.words('indonesian'))
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words_english and word.lower() not in stop_words_indonesian]
    return ' '.join(words)

FUNGSI PREPROSESING DATA

In [7]:
data['cleaned_text'] = data['job_description'].apply(preprocess_text)

MELAKUKAN PREPORSING

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['cleaned_text'])

MERUBAH KE BENTUK VEKTOR

In [ ]:
# Hitung cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

COSINE SIMILARITY

In [ ]:
# Fungsi pencarian pekerjaan# Fungsi pencarian pekerjaan
def search_jobs(query, cosine_sim, data):
    # Preprocess query
    query = preprocess_text(query)

    # Transform query menjadi vektor TF-IDF
    query_vector = vectorizer.transform([query])

    # Hitung similarity antara query dan dokumen
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix)

    # Ambil 5 pekerjaan dengan similarity tertinggi
    top_jobs_indices = similarity_scores.argsort()[0][-5:][::-1]

    # Urutkan hasil berdasarkan similarity score
    sorted_results = sorted([(index, similarity_scores[0][index]) for index in top_jobs_indices], key=lambda x: x[1])

    # Tampilkan hasil beserta similarity score
    results = []
    for index, similarity_score in sorted_results:
        job_title = data.loc[index, 'job_title']
        job_description = data.loc[index, 'job_description']
        results.append({
            'Job Title': job_title,
            'Job Description': job_description,
            'Similarity Score': similarity_score
        })

    return results

FUNGSI MENCARI SIMILARITY QUERY DENGAN YANG AWAL TADI

In [ ]:
example_query = "programmer"

# Lakukan pencarian
search_results = search_jobs(example_query, cosine_sim, data)

# Tampilkan hasil
if isinstance(search_results, str):
    print(search_results)
else:
    for result in search_results:
        print(f"Job Title: {result['Job Title']}")
        print(f"Job Description: {result['Job Description']}")
        print(f"Similarity Score: {result['Similarity Score']:.4f}")
        print("\n")

Job Title: IT Programmer
Job Description: Kami sebagai perusahaan Distribusi Sediaan Farmasi yang sedang berkembang pesat saat ini membutuhkan:IT ProgrammerYang bercita-cita untuk bergerak maju, belajar dan memperdalam keahlian dibidangnya serta berkembang bersama perusahaan kami.Bila anda adalah seseorang dengan :Latar belakang pendidikan IT / SI / Programmer / Software Development yang suka dengan tantangan baru dan mampu bekerja dalam deadline, maka kami adalah perusahaan yang tepat untuk melabuhkan karir anda.Alumni Univ. Bina Nusantara akan memiliki poin lebih, namun terbuka pula untuk lulusan baru universitas lainnya dengan kemampuan dalam pemrograman : PHP, MySQL, HTML, JavaScript, CSS.Lokasi penempatan di Grogol, Jakarta Barat, pusat kota dengan akses yang sangat mudah.Kami mengundang anda untuk bergabung dan berkembang bersama perusahaan kami dalam era digital yang semakin pesat ini.
Similarity Score: 0.1382


Job Title: IT Quality Assurance
Job Description: Customer kami Peru